In [1]:
import sys
sys.path.append('..')

from data_parsers import databento_file_parser
from executor import SimpleExecutor1

import pandas as pd
import math
import numpy as np

from tqdm import tqdm
import math
import numpy
import matplotlib.pyplot as plt
import random
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

In [2]:
data_file_path = '../data/databento/MSFT/xnas-itch-20230508.mbp-10.dbn.zst'
msft_df = databento_file_parser(data_file_path)
msft_df

,ts_event,action,side,depth,price,size,flags,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_px_05,ask_px_05,bid_sz_05,ask_sz_05,bid_px_06,ask_px_06,bid_sz_06,ask_sz_06,bid_px_07,ask_px_07,bid_sz_07,ask_sz_07,bid_px_08,ask_px_08,bid_sz_08,ask_sz_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-08 14:30:00.000611401,2023-05-08 14:30:00.000447666,A,N,9,308.05,96,130,308.14,308.15,143,69,308.13,308.16,404,234,308.12,308.17,437,58,308.11,308.18,568,209,308.10,308.19,168,284,308.09,308.20,118,584,308.08,308.21,231,158,308.07,308.22,368,133,308.06,308.23,318,66,308.05,308.24,310,181,MSFT
2023-05-08 14:30:00.000649278,2023-05-08 14:30:00.000484930,C,N,0,308.14,100,130,308.14,308.15,43,69,308.13,308.16,404,234,308.12,308.17,437,58,308.11,308.18,568,209,308.10,308.19,168,284,308.09,308.20,118,584,308.08,308.21,231,158,308.07,308.22,368,133,308.06,308.23,318,66,308.05,308.24,310,181,MSFT
2023-05-08 14:30:00.000652058,2023-05-08 14:30:00.000488603,C,N,1,308.13,100,130,308.14,308.15,43,69,308.13,308.16,304,234,308.12,308.17,437,58,308.11,308.18,568,209,308.10,308.19,168,284,308.09,308.20,118,584,308.08,308.21,231,158,308.07,308.22,368,133,308.06,308.23,318,66,308.05,308.24,310,181,MSFT
2023-05-08 14:30:00.000659211,2023-05-08 14:30:00.000495912,C,N,2,308.12,100,130,308.14,308.15,43,69,308.13,308.16,304,234,308.12,308.17,337,58,308.11,308.18,568,209,308.10,308.19,168,284,308.09,308.20,118,584,308.08,308.21,231,158,308.07,308.22,368,133,308.06,308.23,318,66,308.05,308.24,310,181,MSFT
2023-05-08 14:30:00.000665286,2023-05-08 14:30:00.000500861,C,N,3,308.11,100,130,308.14,308.15,43,69,308.13,308.16,304,234,308.12,308.17,337,58,308.11,308.18,468,209,308.10,308.19,168,284,308.09,308.20,118,584,308.08,308.21,231,158,308.07,308.22,368,133,308.06,308.23,318,66,308.05,308.24,310,181,MSFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-08 20:59:53.226886085,2023-05-08 20:59:53.226720397,T,A,0,307.73,100,0,307.73,307.80,200,200,307.65,307.83,25,200,307.60,307.85,32,15,307.59,307.88,26,11,307.50,307.93,11,186,307.42,308.13,1,25,307.09,308.14,40,200,307.06,308.30,40,10,307.02,308.50,4,100,307.00,308.55,276,400,MSFT
2023-05-08 20:59:53.226886085,2023-05-08 20:59:53.226720397,T,A,0,307.73,26,130,307.73,307.80,100,200,307.65,307.83,25,200,307.60,307.85,32,15,307.59,307.88,26,11,307.50,307.93,11,186,307.42,308.13,1,25,307.09,308.14,40,200,307.06,308.30,40,10,307.02,308.50,4,100,307.00,308.55,276,400,MSFT
2023-05-08 20:59:53.226886085,2023-05-08 20:59:53.226720397,C,N,0,307.73,26,130,307.73,307.80,74,200,307.65,307.83,25,200,307.60,307.85,32,15,307.59,307.88,26,11,307.50,307.93,11,186,307.42,308.13,1,25,307.09,308.14,40,200,307.06,308.30,40,10,307.02,308.50,4,100,307.00,308.55,276,400,MSFT


## 1 - 2  Avellaneda and Stoikov Model

In [3]:
# Marco Avellaneda & Sasha Stoikov

''' 
This algo give bid and ask quote based on the Sasha Stoikov and Marco Avellaneda Model

Input:
- mid_price = current mid price
- sigma = market volatility    
- q = current position    
- qmax = maximum position
- gamma = 0.1, risk aversion parameter
- k = 1.5, Market model parameter
'''

def quote_stoikov_avellaneda_b(mid_price, sigma, q, q_max, gamma = 0.1, alpha=1):
    # Option B: Unlimit time horizon
    w = 0.5 * gamma**2 * sigma**2 * (q_max + alpha)**2

    # Optimal quotes
    coef = gamma**2 * sigma**2 / (2*w - gamma**2 * q**2 * sigma**2)

    ra = mid_price + np.log(1+(1-2*q)*coef)/gamma
    rb = mid_price + np.log(1+(-1-2*q)*coef)/gamma

    ra = ra.round(2)
    rb = rb.round(2)
        
    return rb, ra

# quote_stoikov_avellaneda_a(10, 0.1, 1000, 0.5, gamma = 0.05, k = 3)
quote_stoikov_avellaneda_b(10, 0.01, 1, 10, gamma = 0.1, alpha = 10)

0.00020000000000000004
0.0025062656641604013


(9.92, 9.97)

In [4]:
class BOT1(SimpleExecutor1):

    def calculate_bid_ask_price(self, bid_orderbook, ask_orderbook, inventory):
        """
        Compute the bid and ask prices based on orderbook status and inventory
        Needs to be overwritten by the model
        """
        mid_price = (ask_orderbook[0][0] + bid_orderbook[0][0])/2
        bid, ask = quote_stoikov_avellaneda_b(mid_price, self.sigma_1min, inventory, 2000, 
                                              gamma=0.01, alpha=4)
        if self.is_verbose_cnt():
            print(f"{mid_price=} {bid=} {ask=} {self.sigma_1min=:.4f} {self.time_frac_elapsed=:.4f}")

        return bid, ask

# Instantiate and run the bot
bot1 = BOT1(ticker='MSFT')
bot1.run_sim(msft_df)

308.145 nan nan nan 0.0
307.935 307.92 307.95 8.425061331297791e-06 0.0027848206338350156
307.88 307.86 307.9 8.312345173314403e-06 0.00631073339576836
307.73 307.71 307.75 8.160879288376185e-06 0.009702136034683433
307.425 307.41 307.44 8.13485505008741e-06 0.012988352733961154
self.current_ts=Timestamp('2023-05-08 14:36:13.786028187') 
self.BID_SIM_ORDER_BOOK={0: (307.68, 120), 1: (307.67, 149), 2: (307.66, 89), 3: (307.65, 130), 4: (307.64, 71), 5: (307.63, 271), 6: (307.62, 221), 7: (307.61, 414), 8: (307.6, 200), 9: (307.59, 346)} 
self.ASK_SIM_ORDER_BOOK={0: (307.7, 43), 1: (307.71, 277), 2: (307.72, 236), 3: (307.73, 311), 4: (307.74, 261), 5: (307.75, 210), 6: (307.76, 157), 7: (307.77, 314), 8: (307.78, 86), 9: (307.79, 76)} 
self.BID_ALGO_ORDERS={307.67: 13} 
self.ASK_ALGO_ORDERS={307.7: 22} 
self.ALGO_POSITION=625 

307.69 307.67 307.71 8.02717285127233e-06 0.015978260888711012
307.57 307.55 307.59 7.93049134624641e-06 0.01975957739560176
307.65 307.63 307.67 7.7417316430277